In [1]:
import pandas as pd
import plotly.express as px
from script_weather import weather_csv

In [2]:
df_final = weather_csv()

KeyboardInterrupt: 

In [7]:
df_final.head()

,year_quarter,avg_risk_score,code_commune_INSEE,code_departement
0,2014T1,0.26,01001,01
1,2014T1,0.26,01002,01
2,2014T1,0.26,01004,01
3,2014T1,0.26,01005,01
4,2014T1,0.26,01006,01


In [4]:
df_final.to_csv('data_risk_weather.csv', index=False)

In [5]:
df_final.shape

(1559096, 4)

In [6]:
df_final['code_departement'].nunique()

94

In [30]:
df_grouped = df_final.groupby(['code_departement', 'year_quarter'])['avg_risk_score'].mean().reset_index()

px.density_heatmap(
    df_grouped,
    x='year_quarter',
    y='code_departement',
    z='avg_risk_score',
    color_continuous_scale='OrRd',
    title='Score moyen de risque climatique par département et trimestre'
)


In [ ]:
import geopandas as gpd

gdf = gpd.read_file(
    "https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements.geojson"
)

print(gdf.columns)

Index(['code', 'nom', 'geometry'], dtype='object')


In [ ]:
gdf_merged = gdf.merge(df_final, 
                       left_on='code', 
                       right_on='code_departement', 
                       how='inner') \
                .drop(columns='code_departement')
gdf_merged.head()

,code,nom,geometry,year_quarter,avg_risk_score,code_commune_INSEE
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2014T1,0.26,01001
1,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2014T1,0.26,01002
2,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2014T1,0.26,01004
3,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2014T1,0.26,01005
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2014T1,0.26,01006


In [ ]:
fig = px.choropleth(
    gdf_merged,
    geojson=gdf,
    locations='code',            # colonne dans ton DataFrame
    featureidkey='properties.code',    # chemin dans le GeoJSON
    color='avg_risk_score',
    animation_frame='year_quarter',    # variable temporelle
    color_continuous_scale='Reds',
    range_color=(0, 3),                # 0 = Low, 3 = Critical
    labels={'avg_risk_score': 'Risk Score'},
    title='Risk Score by Department Over Time'
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.show()
